# Calculation of the Energy of a Chemical Reaction.

In this exercise the energy of a chemical reaction is calculated. For that:
1. The Geoemtries of both reactants an products needs to be optimised to place them at the bottom of the PES and assure that they correspond to minima of it.
2. The Hessian is computed
3. The Hessian is diagonalised and further processed to compute the vibrational frequencies
4. The frequencies are processed to calculate the thermodynamic properties of each of the chemicals involved in the reaction
5. The thermodynamics of the reaction is then computed

As test reaction the reaction between CH3Cl + Br- -> CH3Br + Cl- reaction is chosen. This is a very classical, textbook reaction in which a group of the molecule, Cl, is substituted by an anion, Br-.

In [ ]:
! pip install pyscf geometric ase

In [5]:
import numpy as np

from pyscf import gto
from pyscf.hessian import thermo

from ase.units import Hartree, mol, kcal, Bohr
 

def gen_mol(coords, basis='def2-svp', output='pyscf.log', charge=0):
     mol = gto.Mole()
     mol.atom = coords 
     mol.basis=basis
     mol.output=output
     mol.verbose = 4
     mol.charge = charge
     mol.build()
     return mol


def run_dft(mol, xc_func = 'b3lyp'):
     mf = mol.KS()
     mf.xc = xc_func
     mf.max_cycle=300
     mf.kernel()
     return mf

def calc_e(coords, basis = 'def2-svp', xc_func = 'b3lyp', charge = 0):
     mol = gen_mol(coords, basis = basis, charge = charge)
     mf = run_dft(mol, xc_func)
     return mf  


def calc_gradient(mf):
     g = mf.Gradients().kernel()
     return g


def calc_hessian(mf):
     h = mf.Hessian().kernel()
     return h

#### Defining Coordinates for Reactants and Products
The code starts by defining the atomic coordinates for reactants and products of a chemical reaction using multi-line strings in XYZ format. This includes the positions of atoms in 3D space.

- **`react_1_coords`**: Coordinates for the first reactant molecule, which is likely a chloroalkane.
- **`react_2_coords`**: Coordinates for the second reactant molecule, which is a single bromine atom.
- **`prod_1_coords`**: Coordinates for the first product molecule, which appears to be a bromoalkane.
- **`prod_2_coords`**: Coordinates for the second product molecule, which is a single chlorine atom.

#### Running DFT Calculations for Reactant 1
```python
mf_r1 = calc_e(react_1_coords)
```
- The function `calc_e` is called with the coordinates of the first reactant.
- `calc_e` uses `gen_mol` to create a PySCF `Mole` object with the specified coordinates, basis set (`def2-svp`), and default charge.
- Then, `run_dft` performs a DFT calculation on this molecule using the B3LYP exchange-correlation functional.
- The result, stored in `mf_r1`, includes the DFT calculation data such as the total energy, molecular orbitals, and electronic density.

In [6]:
react_1_coords = '''
C          0.96703        0.06510        0.06459
Cl         2.73400        0.06509        0.06460
H          0.61001        0.74875       -0.70875
H          0.60999        0.39301        1.04331
H          0.60999       -0.94645       -0.14080
'''

react_2_coords = '''
Br         0.0            0.0             0.0
'''

prod_1_coords = '''
C          1.05469        0.05756        0.05617
Br         2.98956        0.05754        0.05618
H          0.71598        0.90558       -0.54079
H          0.71598        0.15053        1.08906
H          0.71598       -0.88344       -0.37976
'''

prod_2_coords = '''
Cl         0.0            0.0             0.0
'''

mf_r1 = calc_e(react_1_coords)
mf_r1.e_tot

output file: pyscf.log


-499.94631200342246


### Additional Steps
For a comprehensive analysis, similar steps would be taken for the other reactant and the products:
1. **Calculate Energy for Reactant 2**:
   ```python
   mf_r2 = calc_e(react_2_coords)
   ```
   - This runs the DFT calculation for the second reactant, a single bromine atom.

2. **Calculate Energy for Product 1**:
   ```python
   mf_p1 = calc_e(prod_1_coords)
   ```
   - This runs the DFT calculation for the first product molecule, which is a bromoalkane.

3. **Calculate Energy for Product 2**:
   ```python
   mf_p2 = calc_e(prod_2_coords)
   ```
   - This runs the DFT calculation for the second product, a single chlorine atom.

### Calculating Reaction Energies
Once the energies of all reactants and products are obtained, you can calculate the reaction energy:

```python
reaction_energy = (mf_p1.e_tot + mf_p2.e_tot) - (mf_r1.e_tot + mf_r2.e_tot)
print(f'Reaction energy: {reaction_energy} Hartree')
```
- This calculates the reaction energy by subtracting the total energies of the reactants from the total energies of the products.
- The result is in Hartree units, a common energy unit in quantum chemistry.

In [7]:
mf_r2 = calc_e(react_2_coords, charge = -1)
mf_r2.e_tot

overwrite output file: pyscf.log


-2573.926779030482

In [8]:
mf_p1 = calc_e(prod_1_coords)
mf_p1.e_tot

overwrite output file: pyscf.log


-2613.7514462244962

In [9]:
mf_p2 = calc_e(prod_2_coords, charge = -1)
mf_p2.e_tot

overwrite output file: pyscf.log


-460.09249422474187

#### Geometry Optimization of a Molecule

To ensure that the molecule is a the bottom of the potential energy surface and in a true minimum of it, an optimisation of the nuclear coordinates is needed. The code provided performs geometry optimization on a molecule and then runs a DFT calculation on the optimized geometry. Here is a step-by-step explanation of what each part of the code does.

#### Importing the Optimization Function


In [10]:
from pyscf.geomopt.geometric_solver import optimize

- This line imports the `optimize` function from the `pyscf.geomopt.geometric_solver` module.
- The `optimize` function is used to find the equilibrium geometry of a molecule by minimizing its energy.

#### Optimizing the Molecular Geometry

```python
mol_eq = optimize(my_mf, maxsteps=100)
```

- The `optimize` function is called with `my_mf`, which is the result of a previous DFT calculation.
- `maxsteps=100` specifies the maximum number of optimization steps.
- The function returns `mol_eq`, which is the optimized molecular geometry.

#### Running DFT Calculation on the Optimized Geometry
```python
mf_eq = run_dft(mol_eq)
print(mol_eq.atom_coords())
```

- The `run_dft` function (defined previously) is called with `mol_eq`.
- This function performs a DFT calculation on the optimized geometry.
- The result, stored in `mf_eq`, contains the DFT calculation data for the optimized molecule.

This call is mainly performed to generate a `mf` object of the optimised geometry over which further processing work can be done.

Finally, `mol_eq.atom_coords()` returns the coordinates of the atoms in the optimized structure.

In [11]:
mol_r1_min = optimize(mf_r1,maxsteps=100)
mol_r1_min.atom_coords()

geometric-optimize called with the following command line:
/Users/Josete/miniforge3/envs/pyscf/lib/python3.12/site-packages/ipykernel_launcher.py -f /Users/Josete/Library/Jupyter/runtime/kernel-5af4c74c-9c3a-4c37-99e8-be9e0b8fb524.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  ** 

array([[ 1.81532724,  0.1232067 ,  0.12211547],
       [ 5.20675941,  0.1232153 ,  0.12193993],
       [ 1.14245794,  1.42241541, -1.34765009],
       [ 1.14262013,  0.74648643,  1.98219527],
       [ 1.14250336, -1.79927955, -0.26809875]])

Now, as before, a `mf` object for the optimised molecule is created for further processing. The attribute `e_tot` prints the electronic energy of the optimised molecule `mf_r1_min`.

In [12]:
mf_r1_min = run_dft(mol_r1_min)
mf_r1_min.e_tot

-499.9466681476985

### Calculation of the Hessian matrix for thermodynamical analysis

Once the molecules are in the minimum of the PES, the harmonic approximation can be applied and the frequencies of the molecular vibrations can be calculated by the diagnalisation of the Hessian matrix. The Hessian matrix for both reactants is build and stored in `h_r1` and `h_r2`, respectively

In [18]:
h_r1 = calc_hessian(mf_r1_min)

In [19]:
h_r2 = calc_hessian(mf_r2)

### Optimisation and Hessian calculation for the products

The same procedure is repeated for the products. As the second product is only a Cl$^-$ anion, no geometry optimistion is needed. Therefore only the geometry of the first product is optimised and stored in the `mol_p1_min` object. The Hessians for the product will be stored in `h_p1` and `h_p2` respectively.

In [17]:
mol_p1_min = optimize(mf_p1, maxsteps=100)
mol_p1_min.atom_coords()

geometric-optimize called with the following command line:
/Users/Josete/miniforge3/envs/pyscf/lib/python3.12/site-packages/ipykernel_launcher.py -f /Users/Josete/Library/Jupyter/runtime/kernel-5af4c74c-9c3a-4c37-99e8-be9e0b8fb524.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  ** 

array([[ 1.98991251,  0.10879806,  0.10616901],
       [ 5.6803099 ,  0.10871457,  0.10618182],
       [ 1.34254908,  1.71745511, -1.02625799],
       [ 1.34251626,  0.28519417,  2.06550839],
       [ 1.34249373, -1.67622489, -0.72074655]])

In [20]:
mf_p1_min = run_dft(mol_p1_min)
mf_p1_min.e_tot

-2613.7516158144854

In [21]:
h_p1 = calc_hessian(mf_p1_min)

In [22]:
h_p2 = calc_hessian(mf_p2)

### Calculation of the Thermodynamics of the System

In [23]:
def thermodynamics(mf, h, T = 298.15, p = 1, kJ = False):
    pressure = int(p * 101325)
    
    freq_info = thermo.harmonic_analysis(mf.mol, h)
    thermo_info = thermo.thermo(mf, freq_info['freq_au'], T, pressure)
    
    energy_conversion = Hartree * mol/kcal if not kJ else Hartree * mol/kJ 
    
    H = thermo_info['H_tot'][0] * energy_conversion
    G = thermo_info['G_tot'][0] * energy_conversion
    
    return G, H

G_r1, enthalpy_r1 = thermodynamics(mf_r1_min, h_r1)
enthalpy_r1

-313695.2920172037

With the information of the Hessian matrix of the chemical species involved in the reaction, its thermodynamic properties, specifically Gibbs free energy and enthalpy, can be computing by using the PySCF quantum chemistry package. Here's a detailed explanation of each part of the code:

#### Defining the Thermodynamics Function
The `thermodynamics` function calculates the Gibbs free energy (G) and enthalpy (H) of a molecule at a given temperature and pressure using harmonic analysis.

```python
def thermodynamics(mf, h, T = 298.15, p = 1, kJ = False):
```
- **`mf`**: A PySCF mean-field object, which contains information about the molecule after a DFT calculation.
- **`h`**: The Hessian matrix, which is used for vibrational frequency analysis.
- **`T`**: Temperature in Kelvin, defaulting to 298.15 K (standard temperature).
- **`p`**: Pressure in atmospheres, defaulting to 1 atm (standard pressure).
- **`kJ`**: A boolean flag to specify whether the results should be in kJ/mol instead of kcal/mol.

#### Converting Pressure to Pascals
```python
    pressure = int(p * 101325)
```
- Pressure is converted from atmospheres to Pascals because the internal calculations require SI units.

#### Performing Harmonic Frequency Analysis
```python
    freq_info = thermo.harmonic_analysis(mf.mol, h)
```
- This line performs a harmonic frequency analysis on the molecule using the Hessian matrix `h`.
- `freq_info` contains the vibrational frequencies in atomic units.

#### Calculating Thermodynamic Properties
```python
    thermo_info = thermo.thermo(mf, freq_info['freq_au'], T, pressure)
```
- This line calculates various thermodynamic properties using the vibrational frequencies, temperature, and pressure.
- `thermo_info` is a dictionary containing the thermodynamic properties.

#### Energy Conversion Factor
```python
    energy_conversion = Hartree * mol/kcal if not kJ else Hartree * mol/kJ 
```
- The energy conversion factor is set depending on whether the results should be in kcal/mol or kJ/mol.
- `Hartree` is the unit of energy used in quantum chemistry calculations, and it is converted to kcal/mol or kJ/mol as required.

#### Extracting and Converting Enthalpy and Gibbs Free Energy
```python
    H = thermo_info['H_tot'][0] * energy_conversion
    G = thermo_info['G_tot'][0] * energy_conversion
```
- The total enthalpy (`H_tot`) and Gibbs free energy (`G_tot`) are extracted from `thermo_info` and converted to the desired units using the energy conversion factor.

#### Returning the Results
```python
    return G, H
```
- The function returns the Gibbs free energy and enthalpy.

### Using the Thermodynamics Function
The function is called with the mean-field object and Hessian for a specific reactant.

```python
G_r1, enthalpy_r1 = thermodynamics(mf_r1_min, h_r1)
enthalpy_r1
```
- **`mf_r1_min`**: The mean-field object for the optimized geometry of reactant 1.
- **`h_r1`**: The Hessian matrix for reactant 1.
- The function returns the Gibbs free energy (`G_r1`) and enthalpy (`enthalpy_r1`) for reactant 1.
- `enthalpy_r1` is printed or returned as the final result.

Putting all together, the code lookes like:

In [24]:
G_r2, enthalpy_r2 = thermodynamics(mf_r2, h_r2)
enthalpy_r2

-1615161.9577286001

In [25]:
G_p1, enthalpy_p1 = thermodynamics(mf_p1_min, h_p1)
enthalpy_p1

-1640128.1970200848

In [26]:
G_p2, enthalpy_p2 = thermodynamics(mf_p2, h_p2)
enthalpy_p2

-288710.91777976416

With this, the enthalpy of the reaction, `H_r` is obtained by substracting to the enthalpy of the products that of the reactants. The Gibb's free energy of the reaction, `G_r` is obtained in the same way.

In [27]:
H_r = (enthalpy_p1 + enthalpy_p2) - (enthalpy_r1 + enthalpy_r2)
print(f'Reaction enthalpy: {H_r:.1f} kcal/mol') 

Reaction enthalpy: 18.1 kcal/mol


In [28]:
G_r = (G_p1 + G_p2) - (G_r1 + G_r2)
print(f'Reaction Gibbs free energy: {G_r:.1f} kcal/mol') 

Reaction Gibbs free energy: 18.0 kcal/mol


In [26]:
TS_coords = '''
C          1.11301       -1.11634        1.82783
H          1.99710       -1.52477        1.29647
H          0.67412       -0.14780        1.51149
H          0.66748       -1.67628        2.67562
Cl         0.08622       -1.93448        0.74785
Br         2.07972       -0.34655        2.84414
'''

mf_ts = calc_e(TS_coords, charge = -1)
mf_ts.e_tot

overwrite output file: pyscf.log


-3073.5105420050786

In [27]:
params = {'transition': True, 'trust': 0.02, 'tmax': 0.06}
mol_ts_opt = mf_ts.Gradients().optimizer(solver='geomeTRIC').kernel(params)
#mol_ts_opt.atom_coords()

from pyscf.geomopt.geometric_solver import optimize
#mol_ts_opt = optimize(mf_ts, maxsteps=100, geomopt = 'TS', trust = 0.02)
#optimizer.hessian_update(h_internal)

geometric-optimize called with the following command line:
/Users/Josete/miniforge3/envs/pyscf/lib/python3.12/site-packages/ipykernel_launcher.py -f /Users/Josete/Library/Jupyter/runtime/kernel-fabb3ff2-505c-43b0-8cc6-6977b180730a.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  ** 

In [28]:
mf_ts_opt = run_dft(mol_ts_opt)

In [29]:
h_ts = calc_hessian(mf_ts_opt)

In [30]:
freq_info = thermo.harmonic_analysis(mf_ts_opt.mol, h_ts, exclude_trans=False, exclude_rot=False)
freq_info['freq_wavenumber']

array([   0.        +47.05698751j,    0.        +42.73367632j,
          0.         +9.8716741j ,   56.65691976 +0.j        ,
         66.70052504 +0.j        ,   69.88200523 +0.j        ,
        399.99369298 +0.j        ,  400.00790369 +0.j        ,
        404.01973653 +0.j        ,  591.6692814  +0.j        ,
        935.49964068 +0.j        ,  940.85919588 +0.j        ,
       1278.24987834 +0.j        , 1433.17478444 +0.j        ,
       1433.63117202 +0.j        , 3106.47931642 +0.j        ,
       3233.83456432 +0.j        , 3235.3658998  +0.j        ])

In [32]:
import nglview as nv
import numpy as np
import mdtraj as md

def visualise_freqs(mol, freqs, n_frames = 20, amplitude = 3):
    # Number of frames for the vibration

    # Create an array to store the frames
    frames = np.zeros((n_frames, mol.natm, 3))

    # Amplitude of vibration (this can be adjusted for better visualization)
    amplitude = 3

    # Generate frames of the vibrating molecule
    for i in range(n_frames):
        xs = mol.atom_coords(unit='ANG')
        displacement = amplitude * np.sin(2 * np.pi * i / n_frames) * freqs 
        frames[i] = (xs + displacement)
    frames = frames * 0.1 # To convert from  (in pyscf) to nm (for NGLViewer)    

    #print(frames)
    # Create an empty topology and add atoms manually
    topology = md.Topology()
    chain = topology.add_chain()
    residue = topology.add_residue('MOL', chain)
    for idx in range(0, mol.natm):
        symbol = mol.atom_symbol(idx)
        topology.add_atom(symbol, element=md.element.get_by_symbol(symbol), residue=residue)

    # Create a trajectory using mdtraj
    traj = md.Trajectory(frames, topology)

    # Visualize with nglview
    view = nv.show_mdtraj(traj)
    view.add_representation('ball+stick', radiusScale=1, showAxes=True)
    #view.add_representation('axes')
    view.center()

    # Display the view
    return view

freqs = freq_info['norm_mode'][0]

visualizer = visualise_freqs(mf_ts_opt.mol, freqs, n_frames = 20, amplitude = 5)
visualizer

NGLWidget(max_frame=19)

In [33]:
# Save the molecule to an XYZ file
mol_ts_opt.tofile('TS_SN2.xyz', format='xyz')

'6\nXYZ from PySCF\nC           2.08644        0.01093        1.82323\nH           2.63299       -0.88713        1.52258\nH           2.43632        0.34071        2.80545\nH           1.01453       -0.20263        1.86348\nCl          2.38792        1.34737        0.58951\nBr          1.64107       -2.37787        4.05139'